# __Entregable__: Preprocesamiento de datos multivariados o de una serie temporal

Este es el entregable 1 preparado para el módulo de Estadística (Bloque 2) para la concentración de IA Avanzada<br>

Tienes que utilizar las variables explicativas y dependientes de tu ÚLTIMO modelo que hiciste en el entregable del Bloque 1. Recuerda que las variables independientes deben ser razones financieras así como las variables dummy (0/1) de la variable de tamaño ("size").

1. Realiza un análisis exploratorio de las variables:
    1. Calcula matriz de varianza y covarianza, así como matriz de correlación de las variables independientes y la dependiente. Explicar qué es la varianza, covarianza y correlación. Interpreta la matriz de correlación. Tiene que utilizar álgebra matricial y corroborar resultados con funciones de Python.
    2. Corre pruebas estadísticas para detectar outliers y leverage points. Tiene que utilizar álgebra matricial para las pruebas y explicar claramente cómo funcionan las pruebas. Puede utilizar funciones de Python para corroborar resultados.
2. Hace un análisis de multicolinealidad explicando la prueba e implicaciones en el modelo.
3. Propone e implementa soluciones a los problemas de los puntos anteriores para que el modelo sea el más adecuado.
4. Estima e interpreta un modelo de regresión múltiple después de atender los problemas anteriores. Tiene que utilizar álgebra matricial para estimar coeficientes y errores estándar del modelo de regresión, y utilizar funciones de Python para corroborar resultados.

In [2]:
# Data management module
import numpy as np
import pandas as pd
# Visualization modules
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
# Linear regression modules
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats.mstats import winsorize
# Statistic module
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
# quit warnings
import warnings
warnings.filterwarnings("ignore")

# To show graphics in the notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

### Análisis exploratorio de las variables

In [3]:
# Open files and save them into a DataFrame
us = pd.read_csv('us2022q2a.csv') # https://apradie.com/datos/us2022q2a.csv
firms = pd.read_csv('usfirms2022.csv') # https://apradie.com/datos/usfirms2022.csv
print(us.columns) # Check columns in us data
print()
print(firms.columns) # Check columns in firms data

Index(['firm', 'q', 'revenue', 'cogs', 'sgae', 'otheropexp', 'extraincome',
       'finexp', 'incometax', 'totalassets', 'totalliabilities', 'shortdebt',
       'longdebt', 'stockholderequity', 'adjprice', 'originalprice',
       'sharesoutstanding', 'fiscalmonth', 'year', 'cto'],
      dtype='object')

Index(['Ticker', 'Name', 'N', 'Class', 'Country\nof Origin', 'Type of Asset',
       'Sector NAICS\nlevel 1', 'Exchange / Src', 'Sector\nEconomatica',
       'Sector NAICS\nlast available', 'partind'],
      dtype='object')


In [4]:
# Merge us and firms data into us_firms 
us_firms = us.merge(firms, left_on='firm', right_on='Ticker')
# Drop useless columns
us_firms = us_firms.drop(['Sector\nEconomatica','Sector NAICS\nlast available','partind','N', 'year','Exchange / Src','Class','cto',
                          'Country\nof Origin','Type of Asset','Ticker', 'extraincome', 'fiscalmonth'], axis=1)
# Extract Financial services information
us_firms = us_firms[us_firms['Sector NAICS\nlevel 1'].isin(['Finance and Insurance', 'Real Estate and Rental and Leasing'])]
us_firms.head()

,firm,q,revenue,cogs,sgae,otheropexp,finexp,incometax,totalassets,totalliabilities,shortdebt,longdebt,stockholderequity,adjprice,originalprice,sharesoutstanding,Name,Sector NAICS\nlevel 1
180,AAIC,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.840768,10.8750,NaN,Arlington Asset Investment Corp,Finance and Insurance
181,AAIC,2000q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.524712,8.1250,NaN,Arlington Asset Investment Corp,Finance and Insurance
182,AAIC,2000q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.547554,9.3125,NaN,Arlington Asset Investment Corp,Finance and Insurance
183,AAIC,2000q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.231498,6.5625,NaN,Arlington Asset Investment Corp,Finance and Insurance
184,AAIC,2001q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.564359,5.4800,NaN,Arlington Asset Investment Corp,Finance and Insurance


#### Get variable calculations
Variable calculations:
  - Firm size measures:
    + Book value of the firm = (totalassets-totalliabilities)
    + Market value = (originalprice * sharesoutstanding) -> (precio del stock histórico)
    <br><br>
  - Profit Margin measures:
    + OPM (Operating Profit Margin) = operating profit / sales = ebit / revenue
      * Operating profit = (revenue - cogs - sgae - otheropexp) = ebit or Ebit (Earning before Interst and Taxes)
      * Cogs = Cost of Good Sold -> __Variable cost__
      * Sgae = Sales and General Administrative Expenses -> __Fixed costs__
      <br><br>
    * Profit margin = Net income / (revenue = sales)<br>
    * Net income = ebit - incometax - finexp<br>
    * Income tax = what the firm pays in taxes (for the government)(impuesto sobre la renta)<br>
    * Finexp = financial expenses = what the firm pays in interest expenses for any loan that the firm issued<br>
<br>

__Notes__:                      Future<br>
  - Dependet variable => Stock Annual Returns (cc.) 1 Quarter<br>
  |----------------------- Later (F1r) -> returns shifted one<br>
  Catalog (possible) Independent variables<br>
  EPSP = EPS / StockPrice<br>
  EPS = NetIncome / sharesoutstanding<br>
  
  *__SELECT 3__*
  1. Sales annual growth = (revenue_t / revenue(t-4)) - 1
  2. Operating profit growth
      Operating profit = EBIT (Earning Befora Interest and Taxes)
  3. Operating profit margin = EBIT / revenue
  4. Book-to-market ratio = book value / market value = (totalassets - totalliabilities) / (originalprice * sharesoutstanding)
  5. Short financial leverage = shortdebt / totalassets
  6. Long financial leverage = longdebt / totalassets 


In [5]:
# Change ceros for NaN
us_firms['revenue'] = us_firms['revenue'].replace([0], [np.nan])
us_firms['totalassets'] = us_firms['totalassets'].replace([0], [np.nan])
us_firms['originalprice'] = us_firms['originalprice'].replace([0], [np.nan])
us_firms['sharesoutstanding'] = us_firms['sharesoutstanding'].replace([0], [np.nan])

In [6]:
# Firm size measures
us_firms['Ebit'] = us_firms['revenue'] - us_firms['cogs'] - us_firms['sgae'] - us_firms['otheropexp']
us_firms['Net income'] = us_firms['Ebit'] - us_firms['incometax'] - us_firms['finexp']
us_firms['EPS'] = us_firms['Net income'] / us_firms['sharesoutstanding']
us_firms['Book value'] = us_firms['totalassets'] - us_firms['totalliabilities']
us_firms['Market value'] = us_firms['originalprice'] * us_firms['sharesoutstanding']

# 3 or more factors selected + EPSP 
us_firms['EPSP'] = us_firms['EPS']  / us_firms['originalprice']

us_firms['OPM'] = us_firms['Ebit'] / us_firms['revenue']
us_firms['Book-to-market ratio'] = us_firms['Book value'] / us_firms['Market value']
us_firms['Short financial leverage'] = us_firms['shortdebt'] / us_firms['totalassets']
us_firms['Long financial leverage'] = us_firms['longdebt'] / us_firms['totalassets']

us_firms['R'] = np.log(us_firms.groupby(['firm'])['adjprice'].shift(-1)) - np.log(us_firms.groupby(['firm'])['adjprice'].shift(4))
us_firms.head()

,firm,q,revenue,cogs,sgae,otheropexp,finexp,incometax,totalassets,totalliabilities,...,Net income,EPS,Book value,Market value,EPSP,OPM,Book-to-market ratio,Short financial leverage,Long financial leverage,R
180,AAIC,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,AAIC,2000q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,AAIC,2000q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,AAIC,2000q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,AAIC,2001q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.440556


In [7]:
# Get actual information
aux = us_firms['q'] == '2022q1'
us_firms2022 = us_firms[aux]
us_firms2022.head()

# Change na to zeros
us_firms2022 = us_firms2022.fillna(0)

,firm,q,revenue,cogs,sgae,otheropexp,finexp,incometax,totalassets,totalliabilities,...,Net income,EPS,Book value,Market value,EPSP,OPM,Book-to-market ratio,Short financial leverage,Long financial leverage,R
268,AAIC,2022q1,8470.0,4773.0,0.0,0.0,0.0,2287.0,920883.0,702786.0,...,1410.0,0.040267,218097.0,1.215069e+05,0.011604,0.436482,1.794935,NaN,0.178003,-0.217590
448,AAME,2022q1,51608.0,0.0,47812.0,0.0,0.0,954.0,375031.0,248608.0,...,2842.0,0.139460,126423.0,6.378494e+04,0.044556,0.073554,1.982019,NaN,0.089961,-0.310124
898,AAT,2022q1,101470.0,0.0,73128.0,0.0,14666.0,0.0,3011134.0,1805247.0,...,13676.0,0.225961,1205887.0,2.293248e+06,0.005964,0.279314,0.525842,NaN,0.547644,-0.046759
1258,ABCB,2022q1,183374.0,10830.0,0.0,0.0,0.0,27706.0,23560292.0,20553133.0,...,144838.0,2.078709,3007159.0,3.057422e+06,0.047373,0.940940,0.983560,NaN,0.023444,-0.251938
1882,ABR,2022q1,166698.0,82559.0,0.0,0.0,0.0,8188.0,16308906.0,13536858.0,...,75951.0,0.499307,2772048.0,2.595047e+06,0.029268,0.504739,1.068207,NaN,0.802576,-0.092947


In [8]:
us_firms2022 = us_firms2022.sort_values(by='Market value')
us_firms2022 = us_firms2022.reset_index()

us_firms2022["Small"] = us_firms2022.index <= len(us_firms2022) / 3
us_firms2022["Small"] = us_firms2022["Small"].astype(int)

us_firms2022["Medium"] = (us_firms2022.index <= (2 * len(us_firms2022) / 3)) & (us_firms2022.index > (len(us_firms2022) / 3))
us_firms2022["Medium"] = us_firms2022["Medium"].astype(int)

In [9]:
# Matrix with x and y information
Xy = us_firms2022[['EPSP', 'OPM', 'Book-to-market ratio', 'Short financial leverage', 'Long financial leverage', 'Small', 'Medium', 'R']]

# Matrix for X
X = us_firms2022[['EPSP', 'OPM', 'Book-to-market ratio', 'Short financial leverage', 'Long financial leverage', 'Small', 'Medium']]

# Matrix for y
y = us_firms2022['R']

### Calculando matriz de Varianza y Covarianza

La covarianza: es un valor que indica el grado de variación conjunta de dos variables aleatorias respecto a sus medias.<br>
La varianza: es una medida de dispersión que representa la variabilidad de una serie de datos respecto a su media.

- $VarCov(x) =  \frac{1}{N-1}[X'X - \frac{1}{N} (X'1)(X'1)']$

In [10]:
# Calculate the variance-covariance matrix by using the next formula:
# (X' . X - (1/N) . (X' . 1) . (X' . 1)') / (N - 1)
N = len(X)
one = np.ones((N,1))
X_1 = X.T @ X
X_2 = X.T @ one
X_3 = X_2.T
VarCov = (X_1 - (1/N) * (X_2 @ X_3)) / (N-1)
VarCov

var = np.diag(VarCov)
var

,EPSP,OPM,Book-to-market ratio,Short financial leverage,Long financial leverage,Small,Medium
EPSP,0.010510,0.056276,0.008466,0.000141,-0.002644,-0.000667,0.001145
OPM,0.056276,1011.808205,-0.807249,0.010189,0.169977,-0.754498,0.382791
Book-to-market ratio,0.008466,-0.807249,0.405217,0.001662,-0.012417,0.059723,0.011608
Short financial leverage,0.000141,0.010189,0.001662,0.005383,-0.000811,-0.001110,-0.000286
Long financial leverage,-0.002644,0.169977,-0.012417,-0.000811,0.053365,-0.022766,0.011389
Small,-0.000667,-0.754498,0.059723,-0.001110,-0.022766,0.222800,-0.111400
Medium,0.001145,0.382791,0.011608,-0.000286,0.011389,-0.111400,0.222367


array([1.05095032e-02, 1.01180820e+03, 4.05217299e-01, 5.38337810e-03,
       5.33650669e-02, 2.22800173e-01, 2.22366710e-01])

TRAS APLICAR LA FORMULA OBTUVIMOS NUESTRA MATRIZ DE VARIANZA-COVARIANZA, ESTO PORQUE LA DIAGONAL DE LA MATRIZ REPRESENTA LAS VARIANZAS Y LOS EXTREMOS LAS COVARIANZAS.

In [11]:
# Checking if the covariance is correct
covX = X.cov()
covX
# Checking if the variance is correct
varX = np.var(X)
varX

,EPSP,OPM,Book-to-market ratio,Short financial leverage,Long financial leverage,Small,Medium
EPSP,0.010510,0.056276,0.008466,0.000141,-0.002644,-0.000667,0.001145
OPM,0.056276,1011.808205,-0.807249,0.010189,0.169977,-0.754498,0.382791
Book-to-market ratio,0.008466,-0.807249,0.405217,0.001662,-0.012417,0.059723,0.011608
Short financial leverage,0.000141,0.010189,0.001662,0.005383,-0.000811,-0.001110,-0.000286
Long financial leverage,-0.002644,0.169977,-0.012417,-0.000811,0.053365,-0.022766,0.011389
Small,-0.000667,-0.754498,0.059723,-0.001110,-0.022766,0.222800,-0.111400
Medium,0.001145,0.382791,0.011608,-0.000286,0.011389,-0.111400,0.222367


EPSP                           0.010496
OPM                         1010.492459
Book-to-market ratio           0.404690
Short financial leverage       0.005376
Long financial leverage        0.053296
Small                          0.222510
Medium                         0.222078
dtype: float64

AHORA COMPROBAMOS QUE NUESTROS CALCULOS FUERON REALIZADOS CORRECTAMENTE UTILIZANDO LA FUNCIONES QUE POSEE PANDAS PARA OBTENER DICHOS DATOS.

### Calculando matriz de Correlación de las variables independientes y la dependiente

- $Corr(X,y) = \frac{Cov(x,y)}{\sigma_x * \sigma_y}$

In [12]:
# Calculate the corretation matrix by using the next formula:
# Corr(X,y) = Cov(X,y)/std(x)*std(y)
N = len(Xy)
one = np.ones((N,1))
X_1 = Xy.T @ Xy
X_2 = Xy.T @ one
X_3 = X_2.T
VarCovXY = (X_1 - (1/N) * (X_2 @ X_3)) / (N-1)
stdxy = np.diag(VarCovXY)**0.5

std = [[0 for y in range(len(stdxy))] for x in range(len(stdxy))]
for i in range(len(stdxy)):
    for j in range(len(stdxy)):
        std[i][j] = stdxy[i] * stdxy[j]

std = np.array([np.array(s) for s in std])

CorrXY = VarCovXY / std
CorrXY

,EPSP,OPM,Book-to-market ratio,Short financial leverage,Long financial leverage,Small,Medium,R
EPSP,1.000000,0.017258,0.129736,0.018681,-0.111635,-0.013780,0.023692,0.347528
OPM,0.017258,1.000000,-0.039867,0.004366,0.023132,-0.050252,0.025520,0.121736
Book-to-market ratio,0.129736,-0.039867,1.000000,0.035588,-0.084439,0.198766,0.038670,-0.178885
Short financial leverage,0.018681,0.004366,0.035588,1.000000,-0.047834,-0.032046,-0.008254,-0.145099
Long financial leverage,-0.111635,0.023132,-0.084439,-0.047834,1.000000,-0.208782,0.104546,-0.035933
Small,-0.013780,-0.050252,0.198766,-0.032046,-0.208782,1.000000,-0.500487,-0.097036
Medium,0.023692,0.025520,0.038670,-0.008254,0.104546,-0.500487,1.000000,-0.006950
R,0.347528,0.121736,-0.178885,-0.145099,-0.035933,-0.097036,-0.006950,1.000000


TRAS APLICAR LA FORMULA PODEMOS VER LA MATRIZ DE CORRELACIONES (COVARIANZA ESTANDARIZADA).

In [13]:
# Checking if the corretation is correct
corrXy = Xy.corr()
corrXy

,EPSP,OPM,Book-to-market ratio,Short financial leverage,Long financial leverage,Small,Medium,R
EPSP,1.000000,0.017258,0.129736,0.018681,-0.111635,-0.013780,0.023692,0.347528
OPM,0.017258,1.000000,-0.039867,0.004366,0.023132,-0.050252,0.025520,0.121736
Book-to-market ratio,0.129736,-0.039867,1.000000,0.035588,-0.084439,0.198766,0.038670,-0.178885
Short financial leverage,0.018681,0.004366,0.035588,1.000000,-0.047834,-0.032046,-0.008254,-0.145099
Long financial leverage,-0.111635,0.023132,-0.084439,-0.047834,1.000000,-0.208782,0.104546,-0.035933
Small,-0.013780,-0.050252,0.198766,-0.032046,-0.208782,1.000000,-0.500487,-0.097036
Medium,0.023692,0.025520,0.038670,-0.008254,0.104546,-0.500487,1.000000,-0.006950
R,0.347528,0.121736,-0.178885,-0.145099,-0.035933,-0.097036,-0.006950,1.000000


AHORA COMPROBAMOS QUE NUESTROS CALCULOS FUERON REALIZADOS CORRECTAMENTE UTILIZANDO LA FUNCIÓN DE PANDAS EN PYTHON.

### Análisis de multicolinealidad

In [21]:
# Calculate the multicolinearity
vif_data = pd.DataFrame()
vif  = us_firms2022[['EPSP', 'OPM', 'Book-to-market ratio', 'Short financial leverage', 'Long financial leverage', 'Small', 'Medium']]
vif_data["feature"] = vif.columns
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(vif.values, i) for i in range(len(vif.columns))]
print(vif_data)

                    feature       VIF
0                      EPSP  1.074893
1                       OPM  1.004487
2      Book-to-market ratio  2.278003
3  Short financial leverage  1.038463
4   Long financial leverage  1.340914
5                     Small  1.647038
6                    Medium  1.588187


TRAS CALCULAR LA MULTICOLINEARIDAD PODEMOS OBSERVAR LA RELACIÓN QUE TIENEN NUESTRAS VARIABLES INDEPENDIENTES ENTRE ELLAS, AL TENER UN VALOR MENOR A 5, NOS DAMOS CUENTA DE QUE TODAS LAS VARIABLES SELECCIONADAS SON DE VALOR PARA REALIZAR NUESTRA PREDICCIÓN.

### Estimación e interpretación un modelo de regresión múltiple

#### Estimación de coeficientes

Para poder realizar el calculo de nuestros coeficientes usamos la siguiente formula:

- $ (X' \cdot X)^{-1} \cdot (X' \cdot y) $

In [22]:
# Calculate the betas by using the next formula:
# (X' . X)^-1 . X' . y
X['Ones'] = 1                 # Add a column of ones

X_trans = X.T                 # Transpose de X 
X_aux = X_trans @ X           # Make a dot product
X_inv = np.linalg.inv(X_aux)  # Calculate the inverse
X_again = X_inv @ X_trans     # Make a dot product
Betas = X_again @ y           # Make a dot product
print(Betas)

0    1.610232
1    0.001480
2   -0.141035
3   -0.909584
4   -0.064214
5   -0.078565
6   -0.047276
7    0.018780
dtype: float64


#### Estimación de errores